In [1]:
import zipfile
%matplotlib inline 
import numpy as np 
import scipy as sp 
import matplotlib as mpl
import matplotlib.cm as cm 
import matplotlib.pyplot as plt
import pandas as pd 
import sklearn
import seaborn as sns
import os
import warnings
from collections import Counter
warnings.filterwarnings('ignore')

In [2]:
pip install imblearn

Note: you may need to restart the kernel to use updated packages.


In [3]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

In [4]:
# Establish our path
events = os.path.join("Resources", "events.csv")

In [6]:
#Creata our Dataframe
events = pd.read_csv(events)
events

,id_odsp,id_event,sort_order,time,text,event_type,event_type2,side,event_team,opponent,...,player_in,player_out,shot_place,shot_outcome,is_goal,location,bodypart,assist_method,situation,fast_break
0,UFot0hit/,UFot0hit1,1,2,Attempt missed. Mladen Petric (Hamburg) left f...,1,12.0,2,Hamburg SV,Borussia Dortmund,...,NaN,NaN,6.0,2.0,0,9.0,2.0,1,1.0,0
1,UFot0hit/,UFot0hit2,2,4,"Corner, Borussia Dortmund. Conceded by Dennis...",2,NaN,1,Borussia Dortmund,Hamburg SV,...,NaN,NaN,NaN,NaN,0,NaN,NaN,0,NaN,0
2,UFot0hit/,UFot0hit3,3,4,"Corner, Borussia Dortmund. Conceded by Heiko ...",2,NaN,1,Borussia Dortmund,Hamburg SV,...,NaN,NaN,NaN,NaN,0,NaN,NaN,0,NaN,0
3,UFot0hit/,UFot0hit4,4,7,Foul by Sven Bender (Borussia Dortmund).,3,NaN,1,Borussia Dortmund,Hamburg SV,...,NaN,NaN,NaN,NaN,0,NaN,NaN,0,NaN,0
4,UFot0hit/,UFot0hit5,5,7,Gokhan Tore (Hamburg) wins a free kick in the ...,8,NaN,2,Hamburg SV,Borussia Dortmund,...,NaN,NaN,NaN,NaN,0,2.0,NaN,0,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
941003,z5L2OT5E/,z5L2OT5E123,123,92,Lucas Torreira (Sampdoria) wins a free kick in...,8,NaN,2,Sampdoria,Atalanta,...,NaN,NaN,NaN,NaN,0,2.0,NaN,0,NaN,0
941004,z5L2OT5E/,z5L2OT5E124,124,93,"Corner, Sampdoria. Conceded by Andrea Masiello.",2,NaN,2,Sampdoria,Atalanta,...,NaN,NaN,NaN,NaN,0,NaN,NaN,0,NaN,0
941005,z5L2OT5E/,z5L2OT5E125,125,93,Attempt missed. Fabio Quagliarella (Sampdoria)...,1,12.0,2,Sampdoria,Atalanta,...,NaN,NaN,8.0,2.0,0,9.0,1.0,1,3.0,0
941006,z5L2OT5E/,z5L2OT5E126,126,94,Alberto Grassi (Atalanta) wins a free kick on ...,8,NaN,1,Atalanta,Sampdoria,...,NaN,NaN,NaN,NaN,0,4.0,NaN,0,NaN,0


In [7]:
events.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 941008 entries, 0 to 941007
Data columns (total 22 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   id_odsp        941008 non-null  object 
 1   id_event       941008 non-null  object 
 2   sort_order     941008 non-null  int64  
 3   time           941008 non-null  int64  
 4   text           941008 non-null  object 
 5   event_type     941008 non-null  int64  
 6   event_type2    214293 non-null  float64
 7   side           941008 non-null  int64  
 8   event_team     941008 non-null  object 
 9   opponent       941008 non-null  object 
 10  player         880009 non-null  object 
 11  player2        291310 non-null  object 
 12  player_in      51715 non-null   object 
 13  player_out     51738 non-null   object 
 14  shot_place     227458 non-null  float64
 15  shot_outcome   228497 non-null  float64
 16  is_goal        941008 non-null  int64  
 17  location       467066 non-nul

In [8]:
#Show data columns

events.columns

Index(['id_odsp', 'id_event', 'sort_order', 'time', 'text', 'event_type',
       'event_type2', 'side', 'event_team', 'opponent', 'player', 'player2',
       'player_in', 'player_out', 'shot_place', 'shot_outcome', 'is_goal',
       'location', 'bodypart', 'assist_method', 'situation', 'fast_break'],
      dtype='object')

In [9]:
# Find out the values of the "is goal column"
events["is_goal"].unique()

array([0, 1], dtype=int64)

In [10]:
#find out the count of "is goal golumn"
events["is_goal"].value_counts()

0    916562
1     24446
Name: is_goal, dtype: int64

In [11]:
# Reading the dictionary data to see what event type are 

dictionary_text = open('dictionary.txt')

In [12]:
dictionary_text.read()

'event_type\n0\tAnnouncement\n1\tAttempt\n2\tCorner\n3\tFoul\n4\tYellow card\n5\tSecond yellow card\n6\tRed card\n7\tSubstitution\n8\tFree kick won\n9\tOffside\n10\tHand ball\n11\tPenalty conceded\n\n\nevent_type2\n12\tKey Pass\n13\tFailed through ball\n14\tSending off\n15\tOwn goal\n\n\nside\n1\tHome\n2\tAway\n\n\nshot_place\n1\tBit too high\n2\tBlocked\n3\tBottom left corner\n4\tBottom right corner\n5\tCentre of the goal\n6\tHigh and wide\n7\tHits the bar\n8\tMisses to the left\n9\tMisses to the right\n10\tToo high\n11\tTop centre of the goal\n12\tTop left corner\n13\tTop right corner\n\n\nshot_outcome\n1\tOn target\n2\tOff target\n3\tBlocked\n4\tHit the bar\n\n\nlocation\n1\tAttacking half\n2\tDefensive half\n3\tCentre of the box\n4\tLeft wing\n5\tRight wing\n6\tDifficult angle and long range\n7\tDifficult angle on the left\n8\tDifficult angle on the right\n9\tLeft side of the box\n10\tLeft side of the six yard box\n11\tRight side of the box\n12\tRight side of the six yard box\n13\t

In [13]:
encoding = pd.read_table('dictionary.txt',  names=('num','events'))
encoding

,num,events
0,event_type,NaN
1,0,Announcement
2,1,Attempt
3,2,Corner
4,3,Foul
...,...,...
70,situation,NaN
71,1,Open play
72,2,Set piece
73,3,Corner


In [14]:
## checking the events column
events["event_type"].unique()

array([ 1,  2,  3,  8, 10,  7,  9,  4,  6, 11,  5], dtype=int64)

In [15]:
#find out the count of "event_type"
events["event_type"].value_counts()

8     237932
3     232925
1     229134
2      91204
7      51738
9      43476
4      39911
10     10730
11      2706
6       1152
5        100
Name: event_type, dtype: int64

In [16]:
# We need the is_goal  column for events and the event_type when is equal 1 since that is a shot attempt.
shot_attempts = events[events["event_type"]== 1]

# We check the filter
shot_attempts.head() 

,id_odsp,id_event,sort_order,time,text,event_type,event_type2,side,event_team,opponent,...,player_in,player_out,shot_place,shot_outcome,is_goal,location,bodypart,assist_method,situation,fast_break
0,UFot0hit/,UFot0hit1,1,2,Attempt missed. Mladen Petric (Hamburg) left f...,1,12.0,2,Hamburg SV,Borussia Dortmund,...,NaN,NaN,6.0,2.0,0,9.0,2.0,1,1.0,0
11,UFot0hit/,UFot0hit12,12,14,Attempt missed. Shinji Kagawa (Borussia Dortmu...,1,12.0,1,Borussia Dortmund,Hamburg SV,...,NaN,NaN,13.0,2.0,0,15.0,1.0,1,1.0,0
13,UFot0hit/,UFot0hit14,14,17,"Goal! Borussia Dortmund 1, Hamburg 0. Kevin G...",1,12.0,1,Borussia Dortmund,Hamburg SV,...,NaN,NaN,4.0,1.0,1,9.0,2.0,1,1.0,0
14,UFot0hit/,UFot0hit15,15,19,Attempt blocked. Mats Hummels (Borussia Dortmu...,1,NaN,1,Borussia Dortmund,Hamburg SV,...,NaN,NaN,2.0,3.0,0,15.0,1.0,0,1.0,0
17,UFot0hit/,UFot0hit18,18,20,Attempt blocked. Tomas Rincon (Hamburg) right ...,1,NaN,2,Hamburg SV,Borussia Dortmund,...,NaN,NaN,2.0,3.0,0,15.0,1.0,0,1.0,0


In [17]:
### We'll check is the number is the same in both databases
shot_attempts["event_type"].value_counts()

1    229134
Name: event_type, dtype: int64

In [18]:
# According to our dictionary database there are some shot locations that are not recorded (number 19)
print(shot_attempts.player[shot_attempts.location == 19].count()) 

1438


In [19]:
# Let's see the percentage. 
print(shot_attempts.event_type[shot_attempts.location == 19].count()/shot_attempts["event_type"].value_counts()* 100) 

1    0.632817
Name: event_type, dtype: float64


In [20]:
#Since is less than 1% we are going to drop those values, even though it may affect our results, it will have a minimal effect

shot_attempts = shot_attempts[shot_attempts.location != 19]


In [21]:
#cHECK NULL VALUIES
shot_attempts.isna().any()

id_odsp          False
id_event         False
sort_order       False
time             False
text             False
event_type       False
event_type2       True
side             False
event_team       False
opponent         False
player           False
player2           True
player_in         True
player_out        True
shot_place        True
shot_outcome     False
is_goal          False
location         False
bodypart         False
assist_method    False
situation        False
fast_break       False
dtype: bool

In [22]:
#Let's start our training and testing
# Create our features
X = shot_attempts[['time', 'side', 'bodypart', 'location', 'situation', 'assist_method', 'fast_break']]

X = pd.get_dummies(X)

# Create our target
y = shot_attempts['is_goal']


In [23]:
X.head()

,time,side,bodypart,location,situation,assist_method,fast_break
0,2,2,2.0,9.0,1.0,1,0
11,14,1,1.0,15.0,1.0,1,0
13,17,1,2.0,9.0,1.0,1,0
14,19,1,1.0,15.0,1.0,0,0
17,20,2,1.0,15.0,1.0,0,0


In [24]:
from sklearn.model_selection import train_test_split
# YOUR CODE HERE
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)
X_train.shape

(170763, 7)

In [25]:
X.describe()

,time,side,bodypart,location,situation,assist_method,fast_break
count,227684.000000,227684.000000,227684.000000,227684.000000,227684.000000,227684.00000,227684.000000
mean,49.219471,1.445047,1.627725,10.118067,1.273388,1.09218,0.020151
std,26.390338,0.496972,0.741303,5.194692,0.694406,0.93339,0.140516
min,0.000000,1.000000,1.000000,3.000000,1.000000,0.00000,0.000000
25%,27.000000,1.000000,1.000000,3.000000,1.000000,0.00000,0.000000
50%,50.000000,1.000000,1.000000,11.000000,1.000000,1.00000,0.000000
75%,72.000000,2.000000,2.000000,15.000000,1.000000,2.00000,0.000000
max,100.000000,2.000000,3.000000,18.000000,4.000000,4.00000,1.000000


In [26]:
# Check the balance of our target values
y.value_counts()

0    204681
1     23003
Name: is_goal, dtype: int64

In [27]:
# Balanced Random Forest Classifier

# Resample the training data with the BalancedRandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
brfc = BalancedRandomForestClassifier(n_estimators=100, random_state=1)

# Fit
brfc.fit(X_train, y_train)
print(X_train.shape)

(170763, 7)


In [28]:
# Calculated the balanced accuracy score
# YOUR CODE HERE
y_pred = brfc.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.703975765555825

In [30]:
#Calculate confusion matrix
cm = confusion_matrix(y_test, y_pred)
cm

array([[35894, 15276],
       [ 1688,  4063]], dtype=int64)

In [31]:
cm_df = pd.DataFrame(
    cm, index=["Actual No Goal", "Actual goal"], columns=["Predcited_No goal)", "Predicted goal"])
cm_df

,Predcited_No goal),Predicted goal
Actual No Goal,35894,15276
Actual goal,1688,4063


In [32]:
# Print the imbalanced classification report

print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.96      0.70      0.71      0.81      0.70      0.50     51170
          1       0.21      0.71      0.70      0.32      0.70      0.50      5751

avg / total       0.88      0.70      0.71      0.76      0.70      0.50     56921



In [33]:
# List the features sorted in descending order by feature importance

features_rank = sorted(zip(brfc.feature_importances_, X.columns), reverse=True)
for feature in features_rank:
    print(f"{feature[1]}: ({feature[0]})")

time: (0.4344229051888649)
location: (0.38269743095724157)
assist_method: (0.05916741599568484)
situation: (0.059028980330022865)
fast_break: (0.03119322732190283)
bodypart: (0.022688921218904104)
side: (0.010801118987378915)


In [35]:
#easy Ensemble AdaBoost Classifier
from imblearn.ensemble import EasyEnsembleClassifier


In [36]:
pip install scikit-learn==1.0 -U

  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.1.1
    Uninstalling scikit-learn-1.1.1:
      Successfully uninstalled scikit-learn-1.1.1



ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\gotar\\anaconda3\\Lib\\site-packages\\~klearn\\.libs\\vcomp140.dll'
Consider using the `--user` option or check the permissions.



In [37]:
# Train the EasyEnsembleClassifier
# YOUR CODE HERE
eec = EasyEnsembleClassifier(n_estimators=100, random_state=1)
eec.fit(X_train, y_train)

EasyEnsembleClassifier(n_estimators=100, random_state=1)

In [38]:
# Calculated the balanced accuracy score
# YOUR CODE HERE
y_pred = eec.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.7303629311631727

In [40]:
cm = confusion_matrix(y_test, y_pred)
cm

array([[36984, 14186],
       [ 1507,  4244]], dtype=int64)

In [41]:
# Print the imbalanced classification report
# YOUR CODE HERE
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.96      0.72      0.74      0.82      0.73      0.53     51170
          1       0.23      0.74      0.72      0.35      0.73      0.53      5751

avg / total       0.89      0.72      0.74      0.78      0.73      0.53     56921

